In [15]:
import backtrader as bt
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import math
import backtrader.analyzers as btanalyzers
import multiprocessing
from polygon.rest import RESTClient


In [16]:
API_KEY = "uwQtl3txGt5BLbecq7ZbIu0ZbuitCGjc"

# Loading and Preprocessing Historical Price Data

This section of the notebook focuses on **loading** and **preprocessing** historical price data for two Exchange-Traded Funds (ETFs): **GLD** (representing gold) and **SLV** (representing silver). The data is sourced from CSV files and prepared for subsequent analysis or modeling.

---

## **1. Loading Data from CSV Files**

**Concept Overview:**

Loading data from CSV (Comma-Separated Values) files is a fundamental step in data analysis. CSV files are widely used for storing tabular data due to their simplicity and compatibility with various tools and programming languages.

**Purpose:**

The primary goal of this code is to:

1. **Load CSV Files:** Import historical price data for GLD and SLV from their respective CSV files into pandas DataFrames.
2. **Convert 'Date' Column to Datetime:** Ensure that the 'Date' columns in both DataFrames are in a datetime format suitable for time series analysis.
3. **Set 'Date' as Index:** Assign the 'Date' column as the index of each DataFrame to facilitate efficient data manipulation and analysis based on dates.

**Benefits:**

- **Structured Data Loading:** Efficiently imports data into pandas DataFrames, enabling powerful data manipulation and analysis capabilities.
- **Time Series Readiness:** Converting the 'Date' column to datetime and setting it as the index prepares the data for time series operations such as resampling, rolling statistics, and plotting.
- **Data Consistency:** Ensures that both datasets (GLD and SLV) are processed uniformly, maintaining consistency in analysis.




In [17]:
# Load the CSV file into a DataFrame
gld_data = pd.read_csv('GLD-Prices.csv')
gld_data['Date'] = pd.to_datetime(gld_data['Date'])  # Ensure the 'Date' column is datetime
gld_data.set_index('Date', inplace=True)  # Set 'Date' as index

# Load the CSV file into a DataFrame
slv_data = pd.read_csv('SLV-Prices.csv')
slv_data['Date'] = pd.to_datetime(slv_data['Date'])  # Ensure the 'Date' column is datetime
slv_data.set_index('Date', inplace=True)  # Set 'Date' as index


In [18]:
def get_asset_data(ticker: str, start_date: str, end_date: str, plot=False):
    """
    Fetches historical price data for a given asset (e.g., GLD, SLV) between the specified start and end dates.
    
    Args:
        ticker (str): The ticker symbol (e.g., 'GLD', 'SLV').
        start_date (str): Start date in the format 'YYYY-MM-DD'.
        end_date (str): End date in the format 'YYYY-MM-DD'.
        plot (bool): Whether to plot the asset's close prices. Default is False.
        
    Returns:
        pd.DataFrame: DataFrame containing the historical price data for the asset.
    """
    try:
        logging.info(f"Initializing RESTClient for {ticker}...")
        rest_client = RESTClient(API_KEY)

        logging.info(
            f"Fetching aggregate bars data for '{ticker}' from {start_date} to {end_date}...")
        # Fetching daily aggregate data
        response = rest_client.get_aggs(
            ticker, 1, 'day', start_date, end_date, limit=50000
        )

        if not response:
            logging.error(
                f"Received empty response from RESTClient for {ticker}.")
            return None

        logging.info(f"Converting response to DataFrame for {ticker}...")
        data = [
            {
                't': item.timestamp,
                'o': item.open,
                'h': item.high,
                'l': item.low,
                'c': item.close,
                'v': item.volume
            }
            for item in response
        ]
        df = pd.DataFrame(data)
        df['Date'] = pd.to_datetime(df['t'], unit='ms')
        df.set_index('Date', inplace=True)
        df.drop(columns=['t'], inplace=True)

        # Rename columns to match desired format
        df.rename(columns={
            'o': 'Open',
            'h': 'High',
            'l': 'Low',
            'c': 'Close',
            'v': 'Volume'
        }, inplace=True)

        # Convert numeric columns to proper type
        numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        df[numeric_columns] = df[numeric_columns].apply(
            pd.to_numeric, errors='coerce')

        # Optionally plot the data
        if plot:
            plt.figure(figsize=(14, 7))
            plt.plot(df.index, df['Close'],
                     label=f'{ticker} Close Price', color='blue')
            plt.title(f'{ticker} Close Prices ({start_date} to {end_date})')
            plt.xlabel('Date')
            plt.ylabel('Price')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

        # Save DataFrame to CSV
        csv_filename = f'{ticker}-Prices.csv'
        df.to_csv(csv_filename, index=True)
        logging.info(f"{ticker} prices saved to '{csv_filename}'.")

    except Exception as e:
        logging.error(f"An error occurred in get_asset_data: {e}")
        return None

    return df


# Fetch and plot GLD data
gld_data = get_asset_data('GLD', '2010-01-01', '2024-09-01')

# Fetch and plot SLV data
slv_data = get_asset_data('SLV', '2010-01-01', '2024-09-01')

# Customized Pandas Data Feed for Backtrader

This section introduces a **customized data feed** tailored for the Backtrader framework, enabling seamless integration of pandas DataFrames containing historical price data. The customization ensures that the data aligns perfectly with Backtrader's expectations, facilitating accurate backtesting and strategy development.

---

## **1. Defining the Customized Pandas Data Feed Class**

### **a. Overview**

Backtrader, a versatile Python library for backtesting trading strategies, relies on data feeds to ingest historical market data. While Backtrader provides several built-in data feeds, customizing a data feed allows for greater flexibility and compatibility with diverse data sources, especially when working with pandas DataFrames.

### **b. Code Implementation**

In [19]:

class PandasData(bt.feeds.PandasData):
    """
    Customized Pandas Data Feed for Backtrader.
    """
    params = (
        ('datetime', None),  # Use the DataFrame index as datetime
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', -1),  # No open interest data
    )


# Create data feeds for GLD and SLV
data_gld = PandasData(dataname=gld_data)
data_slv = PandasData(dataname=slv_data)

# SARIMAX Trading Strategy for Backtrader

This section introduces the **SARIMAXStrategy**, a sophisticated trading strategy implemented using the **Backtrader** framework. The strategy leverages the **Seasonal Autoregressive Integrated Moving Average with eXogenous regressors (SARIMAX)** model to forecast future prices and make informed trading decisions based on these predictions.

---
    
## **1. Strategy Overview**

### **Concept Overview**

The **SARIMAXStrategy** is designed to predict future price movements of a financial instrument (e.g., stocks, ETFs) using the SARIMAX model. SARIMAX is an extension of the ARIMA model that incorporates seasonality and exogenous variables, making it suitable for modeling time series data with seasonal patterns and external influences.

### **Purpose**

The primary objectives of the SARIMAXStrategy are:

1. **Forecasting:**  
   Utilize the SARIMAX model to predict future closing prices based on historical data.

2. **Automated Trading:**  
   Execute buy and sell orders automatically based on the forecasted price movements, incorporating risk management through stop-loss and take-profit mechanisms.

3. **Optimization:**  
   Fine-tune model parameters and trading thresholds to enhance strategy performance through Backtrader's optimization capabilities.

### **Benefits**

- **Data-Driven Decisions:**  
  Relies on statistical models to make informed trading decisions, reducing emotional biases.

- **Seasonality Handling:**  
  Effectively captures and leverages seasonal patterns in price data for more accurate forecasts.

- **Risk Management:**  
  Incorporates stop-loss and take-profit parameters to manage potential losses and lock in profits.

- **Flexibility:**  
  Allows for parameter optimization to adapt the strategy to different market conditions and instruments.

---
    
## **2. Strategy Parameters**

The **SARIMAXStrategy** class inherits from Backtrader's `bt.Strategy` and defines several parameters that control its behavior:# SARIMAX Trading Strategy for Backtrader

This section introduces the **SARIMAXStrategy**, a sophisticated trading strategy implemented using the **Backtrader** framework. The strategy leverages the **Seasonal Autoregressive Integrated Moving Average with eXogenous regressors (SARIMAX)** model to forecast future prices and make informed trading decisions based on these predictions.

---
    
## **1. Strategy Overview**

### **Concept Overview**

The **SARIMAXStrategy** is designed to predict future price movements of a financial instrument (e.g., stocks, ETFs) using the SARIMAX model. SARIMAX is an extension of the ARIMA model that incorporates seasonality and exogenous variables, making it suitable for modeling time series data with seasonal patterns and external influences.

### **Purpose**

The primary objectives of the SARIMAXStrategy are:

1. **Forecasting:**  
   Utilize the SARIMAX model to predict future closing prices based on historical data.

2. **Automated Trading:**  
   Execute buy and sell orders automatically based on the forecasted price movements, incorporating risk management through stop-loss and take-profit mechanisms.

3. **Optimization:**  
   Fine-tune model parameters and trading thresholds to enhance strategy performance through Backtrader's optimization capabilities.

### **Benefits**

- **Data-Driven Decisions:**  
  Relies on statistical models to make informed trading decisions, reducing emotional biases.

- **Seasonality Handling:**  
  Effectively captures and leverages seasonal patterns in price data for more accurate forecasts.

- **Risk Management:**  
  Incorporates stop-loss and take-profit parameters to manage potential losses and lock in profits.

- **Flexibility:**  
  Allows for parameter optimization to adapt the strategy to different market conditions and instruments.

---
    
## **2. Strategy Parameters**

The **SARIMAXStrategy** class inherits from Backtrader's `bt.Strategy` and defines several parameters that control its behavior:

```python

('forecast_period', 1),          # Number of days to forecast ahead
('model_order', (1, 1, 1)),      # SARIMAX (p,d,q) order
('seasonal_order', (1, 1, 1, 7)),  # SARIMAX (P,D,Q,s) seasonal order
('threshold', 0.05),              # Threshold for making trades
('verbose', False),              # Enable detailed logging
('min_data_points', 30),         # Minimum data points required to fit the model
('stop_loss', 0.05),              # 5% stop loss
('take_profit', 0.1),             # 10% take profit
('fit_interval', 5)               # Interval in which to recompute and fit the model
    


In [20]:
import backtrader as bt
import pandas as pd
import math
from statsmodels.tsa.statespace.sarimax import SARIMAX
import logging


class SARIMAXStrategy(bt.Strategy):
    params = (
        ('forecast_period', 1),          # Number of days to forecast ahead
        ('model_order', (1, 1, 1)),      # SARIMAX (p,d,q) order
        ('seasonal_order', (1, 1, 1, 7)),  # SARIMAX (P,D,Q,s) seasonal order
        ('threshold', 0.05),              # Threshold for making trades
        ('verbose', False),              # Enable detailed logging
        # Minimum data points required to fit the model
        ('min_data_points', 30),
        ('stop_loss', 0.05),  # 5% stop loss
        ('take_profit', 0.1),  # 10% take profit
        ('fit_interval', 5)  # Interval in which to recompute and fit the model
    )

    def __init__(self):
        # Reference to the close prices
        self.dataclose = self.datas[0].close

        # Initialize variables to track orders
        self.order = None
        self.buyprice = None
        self.share_price = math.inf
        self.counter = 0

    def next(self):
        """
        Called for each new data point. Make predictions and execute trades.
        """
        self.counter += 1
        if self.counter < self.params.fit_interval:
            return
        self.counter = 0

        if self.order:
            # Pending order execution
            return

        # Ensure we have enough data to fit the model
        if len(self.dataclose) < self.params.min_data_points:
            return

        # Extract historical close prices
        historical_data = pd.Series(
            list(self.dataclose.get(size=self.params.min_data_points)),
            index=pd.to_datetime(self.data.datetime.get(
                size=self.params.min_data_points))
        )

        # Drop NaNs or other invalid data points
        historical_data = pd.to_numeric(
            historical_data, errors='coerce').dropna()

        try:
            # Initialize and fit the SARIMAX model
            self.model = SARIMAX(
                historical_data,
                order=self.params.model_order,
                seasonal_order=self.params.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            self.model_fit = self.model.fit(disp=False)

            if self.params.verbose:
                print(self.model_fit.summary())

            # Make a forecast
            forecast = self.model_fit.forecast(
                steps=self.params.forecast_period)
            predicted_price = forecast.iloc[-1]

            # Current price
            current_price = self.dataclose[0]

            if self.params.verbose:
                print(
                    f"Predicted Price: {predicted_price}, Current Price: {current_price}")

            # Check stop-loss and take-profit before making new trades
            if self.position:
                # Calculate stop-loss and take-profit levels
                stop_loss_price = self.buyprice * (1 - self.params.stop_loss)
                take_profit_price = self.buyprice * \
                    (1 + self.params.take_profit)

                if current_price <= stop_loss_price:
                    self.order = self.sell()  # Trigger stop-loss
                    if self.params.verbose:
                        print(f"STOP LOSS TRIGGERED at {current_price}")
                elif current_price >= take_profit_price:
                    self.order = self.sell()  # Trigger take-profit
                    if self.params.verbose:
                        print(f"TAKE PROFIT TRIGGERED at {current_price}")
                return

            # Trading logic based on prediction
            if predicted_price > current_price * (1 + self.params.threshold):
                # Buy signal
                if not self.position:
                    self.order = self.buy()
                    self.share_price = current_price
                    if self.params.verbose:
                        print(f"BUY EXECUTED at {current_price}")
            elif predicted_price < current_price * (1 - self.params.threshold):
                # Sell signal (if there is a position)
                if self.position:
                    self.order = self.sell()
                    if self.params.verbose:
                        print(f"SELL EXECUTED at {current_price}")

        except Exception as e:
            logging.error(f"Error during forecasting and trading: {e}")

    def notify_order(self, order):
        """
        Notification for order status changes.
        """
        if order.status in [order.Submitted, order.Accepted]:
            # Order submitted/accepted, nothing to do
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                if self.params.verbose:
                    print(f"BUY ORDER COMPLETED at {self.buyprice}")
            elif order.issell():
                if self.params.verbose:
                    print(f"SELL ORDER COMPLETED at {order.executed.price}")

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            logging.warning(f"Order {order.Status[order.status]}")

        # Reset orders
        self.order = None

    def notify_trade(self, trade):
        """
        Notification for trade status changes.
        """
        if not trade.isclosed:
            return

        if self.params.verbose:
            print(f"OPERATION PROFIT, GROSS {trade.pnl}, NET {trade.pnlcomm}")

# Custom Analyzer: FinalValue for Backtrader

This section introduces the **FinalValue Analyzer**, a custom analyzer tailored for the **Backtrader** framework. The analyzer is designed to **capture the final portfolio value** at the end of a backtesting run, providing valuable insights into the overall performance of a trading strategy.

---
    
## **1. Analyzer Overview**

### **Concept Overview**

In Backtrader, **Analyzers** are specialized components that compute and return various metrics and statistics about the performance of trading strategies. They operate alongside strategies to provide comprehensive performance evaluations without interfering with the strategy's execution flow.

The **FinalValue Analyzer** is a custom analyzer that specifically focuses on recording the **final value** of the portfolio once the backtest concludes. This metric is crucial for assessing the profitability and effectiveness of a trading strategy.

### **Purpose**

The primary objectives of the **FinalValue Analyzer** are:

1. **Capture Final Portfolio Value:**  
   Record the total value of the portfolio at the end of the backtest, providing a clear indicator of the strategy's success.

2. **Facilitate Performance Comparison:**  
   Enable easy comparison of different strategies or parameter configurations based on their final portfolio values.

3. **Enhance Reporting:**  
   Integrate seamlessly with Backtrader's reporting mechanisms, allowing for automated extraction and presentation of the final portfolio value alongside other performance metrics.

### **Benefits**

- **Simplicity:**  
  Provides a straightforward mechanism to obtain the final portfolio value without the need for complex computations or additional data handling.

- **Reusability:**  
  Can be easily integrated into any Backtrader backtesting setup, making it a versatile tool for various trading strategies.

- **Automation:**  
  Automatically captures and stores the final portfolio value, reducing manual effort and potential errors in performance assessment.

- **Integration with Other Analyzers:**  
  Complements other analyzers (e.g., Sharpe Ratio, DrawDown) to offer a holistic view of strategy performance.

---


In [21]:

class FinalValue(bt.Analyzer):
    """
    Custom Analyzer to capture the final portfolio value.
    """
    def __init__(self):
        self.final_value = 0.0

    def stop(self):
        """
        Called at the end of the strategy run.
        """
        self.final_value = self.strategy.broker.getvalue()

    def get_analysis(self):
        """
        Returns the analysis results.
        """
        return {'final_value': self.final_value}


# Backtrader Backtesting Setup with SARIMAXStrategy and Custom Analyzers

This section outlines the setup and execution of a **Backtrader** backtesting environment using the **SARIMAXStrategy**. The configuration includes initializing the Cerebro engine, adding data feeds, setting up strategy parameters for optimization, incorporating analyzers for performance evaluation, configuring broker settings, executing the backtest, and extracting the results for analysis.

---
    
## **1. Overview**

The provided code performs the following key functions:

1. **Initialize Cerebro Engine:** Sets up Backtrader's core engine for running backtests.
2. **Add Data Feeds:** Imports historical price data for financial instruments (e.g., GLD and SLV).
3. **Add and Configure Strategy with Optimization:** Implements the `SARIMAXStrategy` with multiple parameter configurations for optimization.
4. **Add Analyzers:** Integrates performance analyzers to evaluate strategy effectiveness.
5. **Configure Broker Settings:** Sets initial capital and commission structures.
6. **Run the Backtest:** Executes the backtest using specified CPU resources.
7. **Extract and Display Results:** Retrieves and prints performance metrics from each optimized strategy run.
8. **Optional Plotting:** Provides an option to visualize the backtest results. *Side Note: Will not work inside jupyter notebooks.*

---

In [25]:
# Initialize Cerebro engine
cerebro = bt.Cerebro()

# Add data feeds to Cerebro
cerebro.adddata(data_gld, name='GLD')
# Uncomment the next line if you wish to add SLV data
# cerebro.adddata(data_slv, name='SLV')

# Add the SARIMAX strategy to Cerebro

cerebro.optstrategy(
    SARIMAXStrategy,
    forecast_period=[1],
    model_order=[(1, 1, 1)],
    seasonal_order=[(1, 1, 1, 7)],
    threshold=[0.005],          # Adjusted to 0.5% for percentage-based logic
    verbose=[False],            # Set to True for detailed logging
    stop_loss=[0.05, 0.1, 0.15, 0.2],         # 5%, 10%, 15%, 20% stop loss
    take_profit=[0.1, 0.15, 0.2, 0.25],       # 10%, 15%, 20%, 25% take profit
    fit_interval=[5],
)

cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(FinalValue, _name='final_value')  # Add the custom analyzer


# Set initial capital
cerebro.broker.setcash(100000.0)

# Set commission - 0.1% per trade
cerebro.broker.setcommission(commission=0.001)

# Print starting portfolio value
print(f"Starting Portfolio Value: {cerebro.broker.getvalue():.2f}")

# Run the backtest
num_cpus = multiprocessing.cpu_count()
results = cerebro.run(maxcpus=1)

# Print final portfolio value
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")


# Plot the results
cerebro.plot(style='candlestick')

# Iterate over all strategy instances and extract analyzer data
for strat_list in results:
    for strat in strat_list:
        sharpe = strat.analyzers.getbyname('sharpe_ratio').get_analysis()
        drawdown = strat.analyzers.getbyname('drawdown').get_analysis()
        final_value = strat.analyzers.getbyname('final_value').get_analysis()

        print(f"Strategy Parameters: Stop Loss={strat.params.stop_loss}, Take Profit={strat.params.take_profit}")
        print(f"Sharpe Ratio: {sharpe.get('sharperatio', 'N/A')}")
        print(f"Max Drawdown: {drawdown.max.drawdown}%\n")
        print(f"Final Portfoloio Value: {final_value.get('final_value')}%\n")

Starting Portfolio Value: 100000.00


/opt/anaconda3/envs/sarima_forecast/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/sarima_forecast/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/sarima_forecast/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/envs/sarima_forecast/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this metho

KeyboardInterrupt: 

# Identifying the Best Strategy Parameters Based on Final Portfolio Value

This section focuses on **evaluating** and **identifying** the most effective set of parameters for the `SARIMAXStrategy` based on the **final portfolio value** obtained from multiple backtesting runs. By iterating through the results of the backtests, the code compares portfolio performances and selects the strategy configuration that yields the highest final value, along with associated performance metrics like the Sharpe Ratio and Maximum Drawdown.

---
    
## **1. Overview**

The primary objectives of this code are:

1. **Performance Evaluation:**  
   Analyze the results of multiple backtesting runs to determine which strategy parameters lead to the best portfolio performance.

2. **Parameter Optimization:**  
   Identify the optimal combination of strategy parameters (e.g., `forecast_period`, `model_order`, `seasonal_order`, `threshold`, `stop_loss`, `take_profit`) that maximize the final portfolio value.

3. **Risk-Adjusted Performance Assessment:**  
   Evaluate additional performance metrics such as the Sharpe Ratio and Maximum Drawdown to ensure that the selected strategy not only yields high returns but also manages risk effectively.

---


In [10]:
best_value = -(math.inf)
for strat_list in results:
    for strat in strat_list:
        final = strat.analyzers.getbyname('final_value').get_analysis()
        # Get the portfolio value
        portfolio_value = final.get('final_value')

        # Check if this is the best portfolio value so far
        if portfolio_value > best_value:
            best_value = portfolio_value
            best_params = strat.params
            best_sharpe = strat.analyzers.sharpe_ratio.get_analysis()
            best_drawdown = strat.analyzers.drawdown.get_analysis()


print(f"Best Portfolio Value: {best_value:.2f}")
print(f"Best Parameters: Forecast Period: {best_params.forecast_period}, "
      f"Model Order: {best_params.model_order}, "
      f"Seasonal Order: {best_params.seasonal_order}, "
      f"Threshold: {best_params.threshold}, "
      f"Stop Loss: {best_params.stop_loss}, "
      f"Take Profit: {best_params.take_profit}")
print(f"Sharpe Ratio: {best_sharpe.get('sharperatio', 'N/A')}")
print(f"Max Drawdown: {best_drawdown.max.drawdown}%")

NameError: name 'results' is not defined